## Fast ai lesson 2

import packages

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
import inspect


from __future__ import division,print_function
import os, json
from glob import glob
import numpy as np
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots, get_batches, plot_confusion_matrix, get_data

from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers import Input
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


### Part 1

Training a linear model with keras 

In [31]:
#dummy data
x = np.random.randn(30,2)
y = np.dot(x,[2.,3.])+1
print(x[:5])
print(y[:5])

[[-0.7317 -0.0925]
 [ 0.2821 -2.1868]
 [-0.3691  1.1667]
 [-0.5499 -0.3302]
 [-2.4511  0.7209]]
[-0.7409 -4.9962  3.7619 -1.0904 -1.7397]


In [33]:
#define a linear model with keras
lm = Sequential()
lm.add(Dense(1,input_shape=(2,)))
lm.compile(optimizer = SGD(lr = 0.1),loss = 'mse')
print(lm.evaluate(x,y,verbose=0))
lm.fit(x,y,nb_epoch=5,batch_size=1,verbose= 1)
print(lm.evaluate(x,y,verbose=0))

21.6136741638
Epoch 1/5
30/30 [==============================] - 0s - loss: 2.9386      
Epoch 2/5
30/30 [==============================] - 0s - loss: 2.6207e-04     
Epoch 3/5
30/30 [==============================] - 0s - loss: 1.8821e-08     
Epoch 4/5
30/30 [==============================] - 0s - loss: 3.5882e-13     
Epoch 5/5
30/30 [==============================] - 0s - loss: 1.1700e-13     

## Part 2 

Using the output from CNN and use a NN layer

In [34]:
path = '../data/dogscats/'
model_path = path + 'models/'
if not os.path.exists(model_path) : os.mkdir(model_path)

Start with Vgg model
* get the true labels for every image 
* get the 1000 image net category for every image 
* feed the prediction to a simple linear model 

In [40]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:621: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [41]:
batch_size = 100 #reduce if your gpu does not have enough memory 

Pre processing steps and learning bcolz

In [48]:
inspect.getsourcelines(get_batches)

(["def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, batch_size=4, class_mode='categorical',\n",
  '                target_size=(224,224)):\n',
  '    return gen.flow_from_directory(dirname, target_size=target_size,\n',
  '            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)\n'],
 89)

In [56]:
val_batches = get_batches(path + 'valid', shuffle= False, batch_size=batch_size)
batches = get_batches(path + 'train',shuffle = False, batch_size = batch_size)

Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


#### loading data and using bcolz

In [57]:
import bcolz
def save_array(fname, arr): c=bcolz.carray(arr, rootdir=fname, mode='w'); c.flush()
def load_array(fname): return bcolz.open(fname)[:]

inspect.getsourcelines(get_data)

(['def get_data(path, target_size=(224,224)):\n',
  '    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)\n',
  '    return np.concatenate([batches.next() for i in range(batches.nb_sample)])\n'],
 135)

In [58]:
#loading data and saving them into the path 
val_data = get_data(val_batches)
trn_data = get_data(batches)
print(trn_data.shape)
save_array(model_path + 'train_data.bc', trn_data)
save_array(model_path + 'valid_data.bc',val_data)

TypeError: coercing to Unicode: need string or buffer, DirectoryIterator found

In [54]:
val_data = get_data(val_batches)


TypeError: coercing to Unicode: need string or buffer, DirectoryIterator found

In [55]:
path

'../data/dogscats/'